<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Judy/Cov_Test_European_Call_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   8359      0 --:--:-- --:--:-- --:--:--  8359
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 59.0 MB 59 kB/s 
     |████████████████████████████████| 1.0 MB 58.6 MB/s 
+ set +ex
Installation succeeded!


# Test
### Test the outcome(Cov Matrix) of random number generated by jax.random.multivariate_normal

In [2]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack
import numpy as np

In [3]:
numstocks = 3

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 50
drift = jnp.array([0.]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, 1000000)

temp = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (51,))

In [4]:
x = temp.T
np.cov(x)

array([[ 0.04425469, -0.00573592, -0.00737059],
       [-0.00573592,  0.04611741,  0.01045513],
       [-0.00737059,  0.01045513,  0.03116009]])

In [5]:
x.mean(axis=1)

DeviceArray([-0.02124749, -0.00667389, -0.01751985], dtype=float32)

In [6]:
temp2 = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (1000001,))
x2 = temp2.T
np.cov(x2)

array([[ 6.25302041e-02, -2.02351553e-05, -7.77442945e-05],
       [-2.02351553e-05,  6.25338717e-02, -5.66869403e-05],
       [-7.77442945e-05, -5.66869403e-05,  6.25793803e-02]])

In [7]:
x2.mean(axis=1)

DeviceArray([-1.18167816e-04, -1.64705038e-04, -5.16102373e-05], dtype=float32)

### Cov(0.0625)

In [11]:
# now change code such that 'numsteps' does not represent year
# make dt = year / numsteps
# Add r, and notice that noise must have mean 0, not drift, or else it'll give large option prices
# (done)
# after making the changes, the values are still correct

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        #dx =  drift + noise[t,:] # no need to multiply by sigma here because noise generated by cov not corr
        dx2 = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx2)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalue(key, initial_stocks, numsteps, drift, r, cov, K, T):
  return jnp.mean((jnp.maximum(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:]-K,0)) * jnp.exp(-r[0] * T)) # this is assuming 1 stock for testing price (didn't take avg)
  # must use '-1' not 'numsteps', or else grad will be 0

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T):
  return jnp.mean((jnp.maximum(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T)[:,-1,:], axis=1)-K,0)) * jnp.exp(-r[0] * T)) # this is assuming 3 stocks in basket
  # must use '-1' not 'numsteps', or else grad will be 0

numstocks = 3

rng = jax.random.PRNGKey(1)
rng, key = jax.random.split(rng)
numsteps = 50
drift = jnp.array([0.]*numstocks)
r = drift # let r = drift to match B-S

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([100.]*numstocks) # must be float

T = 1.0
K = 110.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
#fast_simple(key, initial_stocks, numsteps, drift, cov)

keys = jax.random.split(key, 1000000)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

#print(initial_stocks) #S
#print(K) #K
print(cov) #sigma
#print(drift) #drift

#################################################################################### values for checking
#S, K, r = drift, sigma, T
# 100, 110, 0, 0.25, 1
# 1 stock price should be around 6.1904
# 3 stock price should be around 2.3767
# 1 stock case (not basket) delta should be around (0.39888 / numstocks)

#S, K, r = drift, sigma, T
# 200, 180, 0.1, 0.4, 2
# 1 stock price should be around 70.3005
# 3 stock price should be around 58.7488
# 1 stock case (not basket) delta should be around (0.7946 / numstocks)
####################################################################################

# option price
# 1 stock
#print(optionvalue(key, initial_stocks, numsteps, drift, r, cov, K, T)) # here numsteps different from T
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T)) # here numsteps different from T

# delta test
# 1 stock
#goptionvalue = jax.grad(optionvalue,argnums=1)
#print(goptionvalue(keys, initial_stocks, numsteps, drift, r, cov, K, T)) # here numsteps different from T

# 3 stock basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T)) # here numsteps different from T

[[0.0625 0.     0.    ]
 [0.     0.0625 0.    ]
 [0.     0.     0.0625]]
2.365369
[0.09370443 0.09377381 0.09371734]


In [15]:
#Verification(Using numpy)
S0 = np.array([100, 100, 100])
K = 110
T = 1
r = 0
n_step = 50
n_path = 10000


def BC2(S0, K, T, r, cov, n_step, n_path):
  sigma = cov[0,0] ** 0.5
  payoff = np.zeros(n_path)
  dt = T/n_step
  for i in range(n_path):
    S = S0
    random = np.random.multivariate_normal(mean = np.array([0, 0, 0]), cov = cov, size = (1, n_step))  #把random提上來一層
    #print(random)
    for j in range(n_step):
      random_step = random[0,j]
      #print(random_step)
      S = S * np.exp((r - (1/2)*sigma**2) * dt + np.sqrt(dt) * random_step)
      S_avg = np.average(S)
    payoff[i] = max(np.average(S) - K, 0)
  price = np.exp(-r * T) * np.average(payoff)
  return price

BC2(S0, K, T, r, cov, n_step, n_path) 

2.413944207910202

## COV(0.0025)

In [20]:
T = 1.0
K = 110.0
drift = jnp.array([0.]*numstocks)
r = drift
cov = jnp.identity(numstocks)*0.05*0.05
print(cov)
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T))
# option price
# 3 stocks basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T)) # here numsteps different from T

[[0.0025 0.     0.    ]
 [0.     0.0025 0.    ]
 [0.     0.     0.0025]]
0.00039977825
[0.0001815 0.0001813 0.0001813]


In [21]:
#Verification(Using numpy)
S0 = np.array([100, 100, 100])
K = 110
T = 1
r = 0
n_step = 50
n_path = 10000
BC2(S0, K, T, r, cov, n_step, n_path) 

array([0.00089079, 0.00089079, 0.00089079], dtype=float32)

## COV(Correlation Between = 0.06)

In [22]:
cov = jnp.array([[0.0625,0.06,0.06],[0.06,0.0625,0.06],[0.06,0.06,0.0625]])
#print(cov)
cor1 = cov[0,1] / (cov[0,0] ** 0.5 * cov[1,1] ** 0.5)
cor2 = cov[0,2] / (cov[0,0] ** 0.5 * cov[2,2] ** 0.5)
cor3 = cov[1,2] / (cov[1,1] ** 0.5 * cov[2,2] ** 0.5)
#print(cov[0,1])
print(cor1,cor2,cor3)

0.96 0.96 0.96


In [23]:
print(cov) #sigma
# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T))
# option price
# 3 stocks basket
goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T)) # here numsteps different from T

[[0.0625 0.06   0.06  ]
 [0.06   0.0625 0.06  ]
 [0.06   0.06   0.0625]]
6.073679
[0.13232702 0.13232821 0.13232294]


In [24]:
#Verification(Using numpy)
S0 = np.array([100, 100, 100])
K = 110
T = 1
r = 0
n_step = 50
n_path = 10000
BC2(S0, K, T, r, cov, n_step, n_path) 

6.170627081502218

In [25]:
cov = jnp.array([[0.0625,-0.03,-0.03],[-0.03,0.0625,-0.03],[-0.03,-0.03,0.0625]])
#print(cov)
cor1 = cov[0,1] / (cov[0,0] ** 0.5 * cov[1,1] ** 0.5)
cor2 = cov[0,2] / (cov[0,0] ** 0.5 * cov[2,2] ** 0.5)
cor3 = cov[1,2] / (cov[1,1] ** 0.5 * cov[2,2] ** 0.5)
#print(cov[0,1])
print(cor1,cor2,cor3)

-0.48 -0.48 -0.48


In [29]:
print(cov) #sigma
# option price
# 3 stocks basket
print(optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T))
# option price
# 3 stocks basket
#goptionvalueavg = jax.grad(optionvalueavg,argnums=1)
#print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T)) # here numsteps different from T

[[ 0.0625 -0.03   -0.03  ]
 [-0.03    0.0625 -0.03  ]
 [-0.03   -0.03    0.0625]]


TypeError: ignored

# COV